In [ ]:
import pandas as pd

# Load bond transaction and master data
trans = pd.read_csv('trans_clean_outlier.csv')
master = pd.read_csv('master_clean.csv')
treasury_yields = pd.read_csv('treasury_yields_clean.csv')

In [ ]:
# Ensure date columns are in datetime format
trans['trd_exctn_dt'] = pd.to_datetime(trans['trd_exctn_dt'], errors='coerce')
master['mtrty_dt'] = pd.to_datetime(master['mtrty_dt'], errors='coerce')
treasury_yields['Date'] = pd.to_datetime(treasury_yields['Date'], errors='coerce')

In [ ]:
trans.head()

,bond_id,bond_sym_id,trd_exctn_dt,cusip_id,company_symbol,bsym,rptd_pr,yld_sign_cd,yld_pt
0,A.GF_2016-06-29,A.GF,2016-06-29,00846UAG6,A,BBG0000HY389,112.616,NaN,1.742764
1,A.GF_2016-07-05,A.GF,2016-07-05,00846UAG6,A,BBG0000HY389,110.868,NaN,2.165730
2,A.GF_2016-07-06,A.GF,2016-07-06,00846UAG6,A,BBG0000HY389,111.831,NaN,1.921135
3,A.GF_2016-07-07,A.GF,2016-07-07,00846UAG6,A,BBG0000HY389,114.547,NaN,1.266410
4,A.GF_2016-07-08,A.GF,2016-07-08,00846UAG6,A,BBG0000HY389,111.628,NaN,1.966896


In [ ]:
master.head()

,sym_cd,cusip_id,COMPANY_SYMBOL,sub_prdct_type,debt_type_cd,cpn_rt,cpn_type_cd,mtrty_dt,dissem,grade
0,ABB3852123,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN
1,ABB3852125,00037BAB8,ABB,CORP,CORP,2.875,FXPV,2022-05-08,Y,NaN
2,ABB3852142,00037BAC6,ABB,CORP,CORP,4.375,FXPV,2042-05-08,Y,NaN
3,ABB4615652,00037BAD4,ABB,CORP,CORP,2.800,FXPV,2020-04-03,Y,NaN
4,ABB4615653,00037BAE2,ABB,CORP,CORP,3.375,FXPV,2023-04-03,Y,NaN


In [ ]:
treasury_yields.head()

,Date,1M,3M,6M,1Y,2Y,3Y,5Y,7Y,10Y,20Y,30Y
0,1962-01-09,NaN,NaN,NaN,3.32,NaN,3.74,3.93,NaN,4.05,4.08,NaN
1,1962-01-10,NaN,NaN,NaN,3.33,NaN,3.75,3.94,NaN,4.07,4.09,NaN
2,1962-01-11,NaN,NaN,NaN,3.33,NaN,3.77,3.94,NaN,4.08,4.08,NaN
3,1962-01-12,NaN,NaN,NaN,3.30,NaN,3.76,3.95,NaN,4.08,4.09,NaN
4,1962-01-15,NaN,NaN,NaN,3.32,NaN,3.79,3.96,NaN,4.10,4.10,NaN


In [ ]:
# Merge 'trd_exctn_dt' from transactions to master based on bond_sym_id and sym_cd
df = master.merge(
    trans[['bond_sym_id', 'trd_exctn_dt']],  # Select only relevant columns from transactions
    left_on='sym_cd',
    right_on='bond_sym_id',
    how='left'
)
df = df.drop(columns=['sym_cd'])

In [ ]:
df.head()

,cusip_id,COMPANY_SYMBOL,sub_prdct_type,debt_type_cd,cpn_rt,cpn_type_cd,mtrty_dt,dissem,grade,bond_sym_id,trd_exctn_dt
0,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-28
1,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-29
2,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-30
3,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-07-01
4,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-07-08


In [ ]:
# Calculate time to maturity in years
df['maturity_duration'] = (df['mtrty_dt'] - df['trd_exctn_dt']).dt.days / 365

In [ ]:
# Define Treasury maturity durations (in years) for matching
treasury_maturities = {
    "1M": 1 / 12,
    "3M": 3 / 12,
    "6M": 6 / 12,
    "1Y": 1,
    "2Y": 2,
    "3Y": 3,
    "5Y": 5,
    "7Y": 7,
    "10Y": 10,
    "20Y": 20,
    "30Y": 30
}

# Function to find the nearest Treasury maturity
def find_nearest_maturity(duration):
    if pd.isna(duration):  # Handle NaN durations
        return None
    nearest = min(treasury_maturities.keys(), key=lambda x: abs(treasury_maturities[x] - duration))
    return nearest

In [ ]:
# Match each bond's maturity_duration to the nearest Treasury maturity
df['nearest_treasury'] = df['maturity_duration'].apply(find_nearest_maturity)

In [ ]:
df.head()

,cusip_id,COMPANY_SYMBOL,sub_prdct_type,debt_type_cd,cpn_rt,cpn_type_cd,mtrty_dt,dissem,grade,bond_sym_id,trd_exctn_dt,maturity_duration,nearest_treasury
0,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-28,0.860274,1Y
1,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-29,0.857534,1Y
2,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-30,0.854795,1Y
3,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-07-01,0.852055,1Y
4,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-07-08,0.832877,1Y


In [ ]:
# Map the nearest Treasury maturity to the yield
def get_treasury_yield(row, treasury_data):
    # Match transaction date to Treasury yield date
    yield_row = treasury_data[treasury_data['Date'] == row['trd_exctn_dt']]
    if not yield_row.empty:
        maturity_col = row['nearest_treasury']
        return yield_row.iloc[0][maturity_col] if maturity_col in yield_row.columns else None
    return None

In [ ]:
# Apply the mapping function
df['credit_spread'] = df.apply(get_treasury_yield, treasury_data=treasury_yields, axis=1)

In [ ]:
df.head()

,cusip_id,COMPANY_SYMBOL,sub_prdct_type,debt_type_cd,cpn_rt,cpn_type_cd,mtrty_dt,dissem,grade,bond_sym_id,trd_exctn_dt,maturity_duration,nearest_treasury,credit_spread
0,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-28,0.860274,1Y,0.45
1,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-29,0.857534,1Y,0.46
2,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-06-30,0.854795,1Y,0.45
3,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-07-01,0.852055,1Y,0.45
4,00037BAA0,ABB,CORP,CORP,1.625,FXPV,2017-05-08,Y,NaN,ABB3852123,2016-07-08,0.832877,1Y,0.48


In [ ]:
df.to_csv('master_with_credit_spread.csv', index=False)